In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# Explanation of interpretation
#subm_summary = pd.read_csv("/mnt/c/Users/ashleyconard/Downloads/submission_summary.txt", sep="\t", skiprows=15, header=0)
#count = (~subm_summary['ExplanationOfInterpretation'].str.contains('-')).sum()

gencc_s = pd.read_csv("/mnt/c/Users/ashleyconard/Downloads/gencc-submissions.csv", header=0) # skiprows=15
out_dir = "/mnt/c/Users/ashleyconard/Desktop/gencc_summaries/"


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/ashleyconard/Downloads/gencc-submissions.csv'

In [3]:
gencc_s.head()

NameError: name 'gencc_s' is not defined

In [ ]:

# Replace submitted_as_pmids column blank values with NaN
gencc_s['submitted_as_pmids'] = gencc_s['submitted_as_pmids'].replace('', np.nan)
    
# Remove rows where 'submitted_as_pmids' is NaN
gencc_s = gencc_s.dropna(subset=['submitted_as_pmids'])

# Replace all occurrences of 'LIMITED' with 'Limited' and 'Definitive ' with 'Definitive'
gencc_s['submitted_as_classification_name'] = gencc_s['submitted_as_classification_name'].replace({'LIMITED': 'Limited', 'Definitive ': 'Definitive', 'Modeare': 'Moderate', ' Moderate': 'Moderate', 'Moderate ': 'Moderate', 'Disputed evidence': 'Disputed Evidence', 'Disputed': 'Disputed Evidence', 'STRONG': 'Strong', 'Strong ': 'Strong', ' Strong': 'Strong'})

In [ ]:
def create_palette(data, colors):
    unique_categories = data['submitted_as_classification_name'].unique()
    palette = {}
    for category in unique_categories:
        if category in colors:
            palette[category] = colors[category]
        else:
            palette[category] = 'grey'  # Default color
    return palette

# Increase the default font size
sns.set(font_scale=1.2)

# Group by 'gene_symbol' and 'submitted_as_classification_name', and count unique 'disease_title'
grouped_df = gencc_s.groupby(['gene_symbol', 'submitted_as_classification_name'])['disease_title'].nunique().reset_index(name='unique_disease_count')

color_dict = create_palette(grouped_df, {'Limited': 'red', 'Definitive': 'blue'})

plt.figure(figsize=(20, 10))
sns.violinplot(x='submitted_as_classification_name', y='unique_disease_count', data=grouped_df, palette=color_dict)
plt.xticks(rotation=5)
plt.xlabel('Submitter classification Name')
plt.ylabel('Number of unique diseases per gene')
plt.title("GenCC number of unique diseases per gene by submitter classification name")

plt.show()

In [ ]:
grouped_df[grouped_df['submitted_as_classification_name'] == 'Disputed Evidence']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming gencc_s is your DataFrame and it's already loaded

# Convert 'submitted_as_date' to datetime and extract the year
gencc_s['submitted_as_date'] = pd.to_datetime(gencc_s['submitted_as_date'])
gencc_s['year'] = gencc_s['submitted_as_date'].dt.year

# Create a range of years from 2011 to 2024
years = pd.RangeIndex(2011, 2025)

# Remove rows with 'NaN' in 'submitted_as_classification_name' column
gencc_s = gencc_s.dropna(subset=['submitted_as_classification_name'])

# Create a color dictionary
color_dict = {
    'Definitive': (0, 0.3, 0),
    'Strong': 'forestgreen',
    'Moderate': (0, 1, 0),
    'Supportive': 'lightblue',
    'Limited': 'lightsalmon',
    'Disputed Evidence': 'red',
    'No Known Disease Relationship': 'grey'
}

# Create a figure with a subplot for each classification in color_dict
fig, axs = plt.subplots(len(color_dict), 1, figsize=(10, 5*len(color_dict)))

# Plot a bar plot of the year for each classification in color_dict
for i, classification in enumerate(color_dict.keys()):
    classification_df = gencc_s[gencc_s['submitted_as_classification_name'] == classification]
    color = color_dict.get(classification, 'grey')  # Use color from dictionary if it exists, else use 'grey'
    classification_df['year'].value_counts().reindex(years, fill_value=0).sort_index().plot.bar(ax=axs[i], color=color)
    axs[i].set_title(f'{classification} Classification')
    axs[i].set_xticklabels(years, rotation=0)  # Set x-axis tick rotation

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming gencc_s is your DataFrame and it's already loaded

# Convert 'submitted_as_date' to datetime and extract the year
gencc_s['submitted_as_date'] = pd.to_datetime(gencc_s['submitted_as_date'])
gencc_s['year'] = gencc_s['submitted_as_date'].dt.year

# Create a range of years from 2011 to 2024
years = pd.RangeIndex(2011, 2025)

# Remove rows with 'NaN' in 'submitted_as_classification_name' column
gencc_s = gencc_s.dropna(subset=['submitted_as_classification_name'])

# Get all unique classifications
unique_classifications = gencc_s['submitted_as_classification_name'].unique()

# Create a color dictionary
color_dict = {'Limited': 'red', 'Definitive': 'blue'}

# Create a figure with a subplot for each unique classification
fig, axs = plt.subplots(len(unique_classifications), 1, figsize=(10, 5*len(unique_classifications)))

# Plot a bar plot of the year for each unique classification
for i, classification in enumerate(unique_classifications):
    classification_df = gencc_s[gencc_s['submitted_as_classification_name'] == classification]
    color = color_dict.get(classification, 'grey')  # Use color from dictionary if it exists, else use 'grey'
    classification_df['year'].value_counts().reindex(years, fill_value=0).sort_index().plot.bar(ax=axs[i], color=color)
    axs[i].set_title(f'{classification} Classification')
    axs[i].set_xticklabels(years, rotation=0)  # Set x-axis tick rotation

plt.tight_layout()
plt.show()

In [ ]:
# Assuming gencc_s is your DataFrame and it's already loaded

# Plot a histogram of the 'submitter_title' column
gencc_s['submitter_title'].value_counts().plot(kind='bar', color='grey')

plt.title('Occurrence Frequency of Submitter')
plt.xlabel('Submitter Title')
plt.ylabel('Frequency')

plt.xticks(rotation=90)
plt.show()

In [ ]:
# Assuming gencc_s is your DataFrame and it's already loaded

# Filter the DataFrame
filtered_df = gencc_s[gencc_s['submitted_as_classification_name'].isin(['Limited'])]

# Plot a histogram of the 'submitter_title' column
filtered_df['submitter_title'].value_counts().plot(kind='bar', color='red')

plt.title('Occurrence Frequency of Submitter Titles')
plt.xlabel('Submitter Title')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# Assuming gencc_s is your DataFrame and it's already loaded

# Create a color dictionary
color_dict = {
    'Definitive': (0, 0.3, 0),
    'Strong': 'forestgreen',
    'Moderate': (0, 1, 0),
    'Supportive': 'lightblue',
    'Limited': 'lightsalmon',
    'Disputed Evidence': 'red',
    'No Known Disease Relationship': 'grey'
}

# Group by 'submitter_title' and 'submitted_as_classification_name', count the size, unstack and fill NaN with 0
grouped_df = gencc_s.groupby(['submitter_title', 'submitted_as_classification_name']).size().unstack(fill_value=0)

# Sort the columns based on the order in color_dict
grouped_df = grouped_df[list(color_dict.keys())]

# Plot a stacked bar plot with the colors from color_dict
grouped_df.plot(kind='bar', stacked=True, color=list(color_dict.values()), figsize=(10, 7))

plt.title('Occurrence Frequency of Submitter Titles')
plt.xlabel('Submitter Title')
plt.ylabel('Frequency')

plt.xticks(rotation=20, ha='right')
plt.show()